In [1]:
import urllib.request
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
import csv
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from ipywidgets import widgets, interactive
import time
import re

In [5]:
ser = Service("./chromedriver")
op = webdriver.ChromeOptions()
browser = webdriver.Chrome(service=ser, options=op)

game8_char = "https://game8.co/games/fire-emblem-heroes/archives/242267"
browser.get(game8_char)
respData = browser.page_source
browser.close()
nsoup = BeautifulSoup(respData, 'html.parser')
n_table = nsoup.find("table", attrs={'class':'a-table a-table a-table tablesorter'})

In [32]:
n_table.tbody.findAll('tr')[2].a["href"]

'https://game8.co/games/fire-emblem-heroes/archives/330353'

In [33]:
float(n_table.tbody.findAll('tr')[2].b.get_text())

9.9

In [35]:
heroes = []
links = []
ratings = []

for tr in n_table.tbody.findAll("tr"):
    heroes.append(tr.a.get_text()[1:])
    links.append(tr.a["href"])
    try:
        ratings.append(float(tr.b.get_text()))
    except ValueError:
        ratings.append(None)

dfn = pd.DataFrame(heroes, columns = ["Name"])
dfn['Link'] = links
dfn["Rating"] = ratings

In [341]:
dfn.head()

,Name,Link,Rating
0,Ninja Lyn,https://game8.co/games/fire-emblem-heroes/arch...,9.9
1,Brave Hector,https://game8.co/games/fire-emblem-heroes/arch...,9.9
2,Fallen Edelgard,https://game8.co/games/fire-emblem-heroes/arch...,9.9
3,Ninja Corrin (F),https://game8.co/games/fire-emblem-heroes/arch...,9.9
4,Legendary Dimitri,https://game8.co/games/fire-emblem-heroes/arch...,9.5


In [44]:
ser = Service("./chromedriver")
op = webdriver.ChromeOptions()
# below two lines allow us to initiate "Headless Mode", which means that selenium works silently
op.add_argument('--headless')
op.add_argument('--disable-gpu')
browser = webdriver.Chrome(service=ser, options=op)

all_char = []

for needed in links:

    browser.get(needed)
    chardata = browser.page_source
    
    # has all of the html data
    soupy = BeautifulSoup(chardata, 'html.parser')
    # this gets us only the needed html data
    to_add = soupy.find("div", {'class': "l-3colMain__center l-3colMain__center--shadow"})
    all_char.append(to_add)
        
    # buffer for web driver
    time.sleep(5)
    
    #group.findAll("td")[12] is total
browser.quit()

In [98]:
all_char[-7].find("h3", {"id": 'hm_1'}).text.split(":")[0].split(" - ")

['Lyn', 'Lady of the Plains']

In [105]:
len(all_char[0].find("div", {"class": "a-outline"}).findAll("li"))

10

In [107]:
maxi = 0
maxi_char = []
mini = 100000000
mini_char = []

for char in all_char:
    who = char.find("h2", {"id": 'hl_1'}).text.split(" Rating ")[0]
    length = len(char.find("div", {"class": "a-outline"}).findAll("li"))
    if length == maxi:
        maxi_char.append(who)
    if length > maxi:
        maxi = length
        maxi_char = [who]
    if length == mini:
        mini_char.append(who)
    if length < mini:
        mini = length
        mini_char = [who]
print("Max:", maxi)
print(maxi_char)
print("Min:", mini)
print(mini_char)

Max: 12
['Brave Ike', 'Hector', 'Lucina', 'Eirika', 'Lyn']
Min: 9
["Young L'Arachel", 'Winter Altina', 'New Year Peony', 'Elimine', "Valentine's Gustav", 'Winter Lysithea', 'Pirate Surtr', 'Otr', 'Bridal Catria', 'Thórr', 'Brave Eirika', "Valentine's Faye", 'Brave Marianne', 'Ash', 'Sara', 'Flayn', 'Selena', 'Ninja Hana', 'Hoshidan Summer Micaiah', 'Bridal Nailah', 'Ingrid', 'Bridal Micaiah', 'Dedue', 'Marianne', 'Halloween Robin (F)', 'Halloween Tiki', 'Winter Bernadetta', 'Young Tana', 'Eitri', 'Winter Felix', 'Duo Eirika', 'Young Innes', 'Summer Caeda', 'Summer Freyr', 'Leila', 'Plegian Dorothea', 'Tibarn', 'Ascended Laegjarn', 'Ullr', 'Astrid', 'Volke', 'Kris (M)', 'Larcei', 'Kris (F)', 'Summer Mia', 'Duo Ephraim', 'Fallen Corrin (M)', 'Ninja Laevatein', 'Ayra', 'Legendary Byleth (F)', "Valentine's Henriette", 'Rennac', 'Ascended Joshua', 'Nagi', 'Yuri', 'Constance', 'Hapi', 'Múspell', 'Valentian Palla', 'Zeke', 'Winter Manuela', 'Winter Artur', 'Winter Mirabilis', 'Spring Severa',

In [ ]:
<a class="a-btn" href="/games/fire-emblem-heroes/archives/267867" style="border-color: ; background-color: ; color: ;"><span>Weapon Refinery Tier List</span></a>

In [129]:

(all_char[-7].find("div", {"class": "a-outline"}).find("li", text = "Best Weapon Refine") is not None) & (
all_char[-7].find("div", {"class": "a-outline"}).find("li", text = "Lyn " + "Best Weapon Refine") is not None)

False

In [119]:
t = all_char[-7].find("a", {"href": "/games/fire-emblem-heroes/archives/267867"}) is not None
h = all_char[-7].find("a", {"href": "/games/fire-emblem-heroes/archives/267867"}) is not None
t == h

True

In [142]:
large_char = []
i = 0
for char in all_char:
    who = char.find("h2", {"id": 'hl_1'}).text.split(" Rating ")[0]
    result = (char.find("div", {"class": "a-outline"}).find("li", text = "Best Weapon Refine") is not None) | (
        char.find("div", {"class": "a-outline"}).find("li", text =  who + " Best Weapon Refine") is not None) | (
        char.find("th", text = "Exclusive Weapon Effect from Refining (Special Effect Type)") is not None) | (
        char.find("a", {"href": "/games/fire-emblem-heroes/archives/267867"}) is not None)
    if result == True:
        i += 1
        large_char.append(who)
print(i)

257


In [154]:
other_sub = []
i = 0
for char in all_char:
    who = char.find("h2", {"id": 'hl_1'}).text.split(" Rating ")[0]
    result = (
        char.find("th", text = "Exclusive Weapon Effect from Refining (Special Effect Type)") is not None) | (
        char.find("a", {"href": "/games/fire-emblem-heroes/archives/267867"}) is not None) | (
        char.find('h2', text = who + " Best Weapon Refine") is not None)
    if result == True:
        i += 1
        other_sub.append(who)
print(i)

256


In [147]:
sub_char = []
i = 0
for char in all_char:
    who = char.find("h2", {"id": 'hl_1'}).text.split(" Rating ")[0]
    result = (char.find("div", {"class": "a-outline"}).find("li", text = "Best Weapon Refine") is not None) | (
        char.find("div", {"class": "a-outline"}).find("li", text =  who + " Best Weapon Refine") is not None) | (
        char.find("div", {"class": "a-outline"}).find("li", text = "Best Weapon Refinement") is not None) | (
        char.find("div", {"class": "a-outline"}).find("li", text = who + " Best Weapon Refinement") is not None)
    if result == True:
        i += 1
        sub_char.append(who)
print(i)

249


In [155]:
list(set(large_char) - set(sub_char))

['Ares',
 'Mist',
 'Seth',
 'Laslow',
 'Cain',
 'New Year Camilla',
 'Ayra',
 'Mila',
 'Luke']

In [145]:
list(set(sub_char) - set(large_char))

[]

In [156]:
list(set(large_char) - set(other_sub))

['New Year Alfonse']

In [140]:
i = 0

for char in all_char:
    text = char.find("th", text = "Exclusive Weapon Effect from Refining (Special Effect Type)") is not None
    header = char.find("a", {"href": "/games/fire-emblem-heroes/archives/267867"}) is not None
    if text != header:
        if header == True:
            print(char.find("h2", {"id": 'hl_1'}).text.split(" Rating ")[0])
            print("text:", text)
            print("header:", header)
    if (text is True) & (header is True):
        i += 1
print(i)

Boey
text: False
header: True
Summer Camilla
text: False
header: True
Hector
text: False
header: True
Hostile Springs Camilla
text: False
header: True
Bartre
text: False
header: True
New Year Camilla
text: False
header: True
Spring Kagero
text: False
header: True
Spring Alfonse
text: False
header: True
Gunter
text: False
header: True
188


In [121]:
not_match

['Brave Alm',
 'Nailah',
 'Legendary Tiki (Young)',
 'Walhart',
 'Mila',
 'Legendary Hector',
 'Loki',
 'Legendary Marth',
 'Legendary Robin (F)',
 'Fallen Robin (M)',
 'Brave Eliwood',
 'Brave Micaiah',
 'Bridal Tharja',
 'Summer Tiki (Young)',
 'Eir',
 'Legendary Ephraim',
 "Valentine's Hector",
 'Julius',
 'Kaze',
 'Kliff',
 'Libra',
 'Legendary Lucina',
 'Marisa',
 'Morgan (M)',
 'Winter Ephraim',
 'Owain',
 'Soleil',
 'Boey',
 'Caeda',
 'Sumia',
 'Summer Camilla',
 'Spring Catria',
 'Adrift Corrin (F)',
 'Corrin (F)',
 'Adrift Corrin (M)',
 'Legendary Eirika',
 'Hector',
 'Hríd',
 'Laevatein',
 'Lugh',
 'Hostile Springs Camilla',
 'Quan',
 'Sharena',
 'Bartre',
 'Adrift Camilla',
 'New Year Camilla',
 'Ephraim',
 'Spring Kagero',
 'Kana (M)',
 'Karel',
 'Legault',
 'Linde',
 'Luke',
 'Spring Alfonse',
 'Gunter',
 'Jamke']

In [96]:
all_char[0].find("h2", {"id": 'hl_1'}).text.split(" Rating ")[0]

'Ninja Lyn'

In [80]:
all_char[0].find("h3", text = "Stat Rankings").find_next_sibling().findAll('tr')[6].img.get('alt').split(" ")

['Res', '26']

In [198]:
i = 0
for char in all_char:
    res = char.find("th", text = "Harmonized Skill")
    if res is not None:
        print(char.find("h2", {"id": 'hl_1'}).text.split(" Rating ")[0])
        i += 1
i

Winter Altina
Winter Lysithea
Spring Myrrh
Bridal Catria
Hatari Azura
Halloween Tiki
Summer Caeda
Plegian Dorothea
Summer Mia
Scion Leif
Pirate Veronica


11

In [286]:
for char in all_char:
    if char.find('th', text = "Heroes With the Same Name") is not None:
        for table in char.findAll('table', {'class': 'a-table a-table a-table'}):
            to_break = False
            if table.find("th", text = "Heroes With the Same Name") is not None:
                to_break = True
                for a in char.find('table', {'class': 'a-table a-table a-table'}).findAll('a'):
                    print(a.text[1:])
            print(to_break)
            if to_break is True:
                break
    else:
        print(0)

Brave Lyn
Bridal Lyn
Lyn
Legendary Lyn
Valentine's Lyn
Summer Lyn
True
Hector
Valentine's Hector
Legendary Hector
Halloween Hector
True
Edelgard
Legendary Edelgard
Brave Edelgard
True
Adrift Corrin (F)
Corrin (F)
Summer Corrin (F)
Fallen Corrin (F)
Legendary Corrin (F)
True
Dimitri
Brave Dimitri
Fallen Dimitri
True
L'Arachel
Halloween L'Arachel
True
Altina
True
Valentine's Lilina
Lilina
Summer Lilina
True
Peony
True
Freyja
True
New Year Azura
Performing Azura
Azura
Azura (Young)
Hatari Azura
True
0
Sigurd
Legendary Sigurd
True
0
Alm
Legendary Alm
Valentine's Alm
True
Líf
True
New Year Dagr
True
Spring Veronica
Pirate Veronica
Valentine's Veronica
True
Lysithea
Brave Lysithea
True
0
Myrrh
Halloween Myrrh
True
Surtr
True
0
Claude
Brave Claude
True
Catria
Spring Catria
Valentian Catria
True
New Year Peony
True
New Year Reginn
True
Sigurd
Masquerade Sigurd
True
Legendary Ike
Ike
Valentine's Ike
Fallen Ike
True
0
Red Tome Eirika
Legendary Eirika
Eirika
Winter Eirika
Duo Eirika
True
Faye
Tru

0
Zephiel
True
Groom Marth
Marth
Legendary Marth
Young Marth
Brave Marth
True
Jaffar
True
Kris (M)
Kris (F)
True
Fae
Legendary Fae
True
0
0
Lute
True
Legendary Seliph
True
Joshua
Ascended Joshua
True
Lukas
True
0
0
Genny
True
Valentine's Líf
True
0
Sharena
True
0
Performing Azura
Azura
Azura (Young)
Legendary Azura
Hatari Azura
True
Shigure
True
Winter Hilda
Summer Hilda
True
New Year Azura
Performing Azura
Azura
Legendary Azura
Hatari Azura
True
0
0
Halloween Sophia
True
0
Bridal Caeda
Young Caeda
Summer Caeda
True
Adrift Camilla
New Year Camilla
Spring Camilla
Summer Camilla
Hostile Springs Camilla
Brave Camilla
True
Palla
Young Palla
Valentian Palla
True
0
0
Veronica
Pirate Veronica
Valentine's Veronica
True
0
Summer Takumi
New Year Takumi
Takumi
True
Fallen Takumi
New Year Takumi
Takumi
True
Catria
Valentian Catria
Bridal Catria
True
Alfonse
Spring Alfonse
New Year Alfonse
True
Winter Cecilia
True
Tana
Young Tana
True
0
Celica
Brave Celica
Legendary Celica
True
Tharja
Bridal Tharja

In [184]:
for a in all_char[0].find('table', {'class': 'a-table a-table a-table'}).findAll('a'):
    print(a.text[1:])

Brave Lyn
Bridal Lyn
Lyn
Legendary Lyn
Valentine's Lyn
Summer Lyn


In [218]:
who = all_char[0].find("h2", {"id": 'hl_1'}).text.split(" Rating ")[0]
all_char[0].find('h3', text = "How to Get " + who).findNext("h4").findNext("h4").text

'Featured Articles'

In [222]:
all_char[0].find("h1", {"itemprop": 'name'}).text.split(" Builds ")[0]

'Ninja Lyn'

In [249]:
for char in all_char:
    who = char.find("h1", {"itemprop": 'name'}).text.split(" Builds ")[0]
    print(who)
    if who == "Ótr":
        who = "Otr" # uses this for the rest of the doc
    if who == "Thorr":
        who = "Thórr" # uses this for the rest of the doc
    if who == "Halloween Tiki (Young)":
        who = "Halloween Tiki"
    if who == "Duo Eirika and Ephraim":
        who = "Duo Eirika"
    if who == "Fallen Tiki":
        who = "Fallen Tiki (Young)"
    if who == "Legendary Tiki":
        who = "Legendary Tiki (Young)"
    if who == "Pirate Hinoka and Camilla":
        who = "Pirate Hinoka"
    if who == "Scion Leif and Seliph":
        who = "Scion Leif"
    if who == "Masquerade Eldigan Best":
        who = "Masquerade Eldigan"
    if who == "Fallen Robin":
        who = "Fallen Robin (M)"
    if who == "Lif":
        who = "Líf" # uses this the rest of the doc
    if who == "Young Azura":
        who = "Azura (Young)"
    if who == "Gangrel Best":
        who = "Gangrel"
    if who == "Hot Springs Camilla":
        who = "Hostile Springs Camilla"
    if who == "Summer Rhys Best":
        who = "Summer Rhys"
    if who == "Bridal Tanith":
        who = "Tanith" # only refers to Tanith in this one part
    if who == "Summer Lorenz Best":
        who = "Summer Lorenz"
    if who == "Kiran (Summoner)":
        who = "Kiran: Hero Summoner" # the rest of the doc refers to him as "My Summoner (Kiran)"
    start = char.find('h3', text = "How to Get " + who)
    if who == "Kiran: Hero Summoner":
        start = char.find('h3', text = "How to Obtain " + who)
    get = start.findNext("h4")
    while get.text != "Featured Articles":
        print(get.text)
        get = get.findNext('h4')

Ninja Lyn
Obtained through a Special Summoning Event!
Brave Hector
Obtain Through Regular Summon
Fallen Edelgard
Obtain Through Regular Summon
Ninja Corrin (F)
Obtained through a Special Summoning Event!
Legendary Dimitri
Obtained through a Special Summoning Event!
Young L'Arachel
Obtained through a Special Summoning Event!
Winter Altina
Obtained through a Special Summoning Event!
Legendary Lilina
Obtained through a Special Summoning Event!
New Year Peony
Obtained through a Special Summoning Event!
Summer Freyja
Obtained through a Special Summoning Event!
Legendary Azura
Obtained through a Special Summoning Event!
Elimine
Obtained through a Special Summoning Event!
Masquerade Sigurd
Obtained through a Special Summoning Event!
Valentine's Gustav
Obtained through a Special Summoning Event!
Brave Alm
Obtain Through Regular Summon
Valentine's Líf
Obtained through a Special Summoning Event!
Dagr
Obtained through a Special Summoning Event!
Veronica
Obtain Through Regular Summon
Winter Lysith

Obtain Through Regular Summon
Kliff
Obtain Through Regular Summon
Nyx
Obtain Through Regular Summon
Apotheosis Anna
Obtain Through Regular Summon
Halloween Kurthnaga
Obtained through a Special Summoning Event!
Duma
Obtained through a Special Summoning Event!
Brunnya
Obtained as a Reward in Tempest Trials
Halloween Sophia
Obtained as a Reward in Tempest Trials
Lewyn
Obtain Through Regular Summon
Libra
Obtain Through Regular Summon
Linus
Obtain through a Grand Hero Battle
Asbel
Obtain Through Regular Summon
Farina
Obtain Through Regular Summon
Roshea
Obtain Through Regular Summon
Legendary Lucina
Obtained through a Special Summoning Event!
Brave Lyn
Obtain Through Regular Summon
Legendary Roy
Obtained through a Special Summoning Event!
Legendary Lyn
Obtained through a Special Summoning Event!
Catherine
Obtain Through Regular Summon
Vyland
Obtain Through Regular Summon
Azelle
Obtain Through Regular Summon
Marisa
Obtained as a Reward in Tempest Trials
Lex
Obtain Through Regular Summon
Erin

Obtained through a Special Summoning Event!
Ylgr
Obtain Through Regular Summon
Sigrun
Obtained as a Reward in Tempest Trials
Delthea
Obtain Through Regular Summon
Donnel
Obtain Through Regular Summon
Obtain through a Hero Battle
Halloween Dorcas
Obtained as a Reward in Tempest Trials
Spring Saleh
Obtained as a Reward in Tempest Trials
Draug
Obtain Through Regular Summon
Effie
Obtain Through Regular Summon
Summer Mercedes
Obtained through a Special Summoning Event!
Eirika
Obtain Through Regular Summon
Chad
Obtain Through Regular Summon
Summer Elise
Obtained through a Special Summoning Event!
Valentine's Eliwood
Obtained as a Reward in Tempest Trials
Ephraim
Obtain Through Regular Summon
Tanya
Obtain Through Regular Summon
Galle
Obtain through a Grand Hero Battle
Ethlyn
Obtained as a Reward in Tempest Trials
Faye
Obtain Through Regular Summon
Cynthia
Obtain through a Grand Hero Battle
Felicia
Obtain Through Regular Summon
Obtain through a Hero Battle
Abel
Obtain Through Regular Summon
Fi

In [278]:
games = []

for char in all_char:
    who = char.find("h1", {"itemprop": 'name'}).text.split(" Builds ")[0]
    try:
        for div in char.find('h3', text = "Voice Actor and Illustrator Information").findNext("table").findAll('div'):
            games.append(div.text)
    except AttributeError:
        games.append(None)

set(games)

{'Fire Emblem (The Blazing Blade)',
 'Fire Emblem Awakening',
 'Fire Emblem Echoes: Shadows of Valentia (Gaiden)',
 'Fire Emblem Fates: Conquest and Fire Emblem Fates: Birthright',
 'Fire Emblem Heroes',
 'Fire Emblem: Genealogy of the Holy War',
 'Fire Emblem: Mystery of the Emblem and New Mystery of the Emblem',
 'Fire Emblem: New Mystery of the Emblem',
 'Fire Emblem: Path of Radiance',
 'Fire Emblem: Radiant Dawn',
 'Fire Emblem: Shadow Dragon',
 'Fire Emblem: The Binding Blade',
 'Fire Emblem: The Sacred Stones',
 'Fire Emblem: Thracia 776',
 'Fire Emblem: Three Houses',
 None,
 'Tokyo Mirage Sessions ♯FE'}

In [335]:
for char in all_char:
    try:
        tds = char.find('h4', text = "Stats at Lv 40").findNext("table").findAll("td")
        adj_hp = int(tds[0].text)
        adj_atk = int(tds[1].text)
        adj_spd = int(tds[2].text)
        adj_def = int(tds[3].text)
        adj_rest = int(tds[4].text)
        adj_total = int(tds[5].text)
        
        hp = int(tds[6].text)
        atk = int(tds[7].text)
        spd = int(tds[8].text)
        defense = int(tds[9].text)
        resis = int(tds[10].text)
        total = int(tds[11].text)
    except AttributeError:
        print(char.find("h1", {"itemprop": 'name'}).text.split(" Builds ")[0])
        stats = char.find("h3", text = "Recommended IVs").findNext("table").findAll("td")
        to_add = []
        for i in range(0, len(stats), 3):
            start = stats[i].text.find("(")
            end = stats[i].text.find(")")
            to_add.append(stats[i].text[start + 1: end])
        print(to_add)
        hp = int(to_add[0])
        atk = int(to_add[1])
        spd = int(to_add[2])
        defense = int(to_add[3])
        resis = int(to_add[4])
        total = sum(map(int, to_add))
        
        adj_hp.append(None)
        adj_atk.append(None)
        adj_spd.append(None)
        adj_def.append(None)
        adj_rest.append(None)
        adj_total.append(None)

Kiran (Summoner)
['41', '49', '35', '28', '28']


In [283]:
og = []
uni = []

for char in all_char:
    who = char.find("h1", {"itemprop": 'name'}).text.split(" Builds ")[0]
    try:
        if len(char.find('h3', text = "Voice Actor and Illustrator Information").findNext("table").findAll('div')) > 1:
            games = []
            for div in char.find('h3', text = "Voice Actor and Illustrator Information").findNext("table").findAll('div'):
                games.append(div.text) 
            og.append(games)
        else:
            div = char.find('h3', text = "Voice Actor and Illustrator Information").findNext("table").find('div')
            games = div.text
            og.append(games)
            
        verse = set()
        if type(games) is list:
            for game in games:
                verse.add(universe_dict[game])
            if len(verse) == 1:
                uni.append(verse.pop())
            else:
                uni.append(verse)
        else:
            uni.append(universe_dict[games])
    except AttributeError:
        if who == "Summer Ogma":
            og.append("Fire Emblem: Shadow Dragon")
            uni.append("Archanea")
        if who == "Brave Marth":
            og.append("Fire Emblem: Shadow Dragon")
            uni.append("Archanea")
        if who == "Ninja Igrene":
            og.append("Fire Emblem: The Binding Blade")
            uni.append("Elibe")
        if who == "Kiran (Summoner)": # change to Kiran: Hero Summoner
            og.append("Fire Emblem Heroes")
            uni.append("Heroes")

uni

['Elibe',
 'Elibe',
 'Fódlan',
 'Fates',
 'Fódlan',
 'Magvel',
 'Tellius',
 'Elibe',
 'Heroes',
 'Heroes',
 'Fates',
 'Tellius',
 'Jugdral',
 'Heroes',
 'Valentia',
 'Heroes',
 'Heroes',
 'Heroes',
 {'Fódlan', 'Magvel'},
 'Heroes',
 {'Magvel', 'Ylisse'},
 'Heroes',
 'Heroes',
 'Fódlan',
 'Valentia',
 'Heroes',
 'Heroes',
 'Jugdral',
 'Tellius',
 'Heroes',
 'Magvel',
 'Valentia',
 'Fódlan',
 'Fódlan',
 'Heroes',
 'Fódlan',
 'Ylisse',
 'Jugdral',
 'Heroes',
 'Fódlan',
 'Fates',
 'Magvel',
 'Tellius',
 'Fates',
 'Tellius',
 'Tellius',
 'Fódlan',
 'Tellius',
 'Fódlan',
 'Fódlan',
 'Jugdral',
 'Ylisse',
 {'Archanea', 'Elibe'},
 'Tellius',
 'Fates',
 'Fódlan',
 'Magvel',
 'Jugdral',
 'Heroes',
 'Fódlan',
 'Heroes',
 'Magvel',
 'Magvel',
 'Heroes',
 'Heroes',
 'Elibe',
 'Fódlan',
 'Tellius',
 'Tellius',
 'Heroes',
 'Jugdral',
 'Tellius',
 'Elibe',
 'Tellius',
 'Archanea',
 'Fódlan',
 'Jugdral',
 'Fates',
 'Archanea',
 'Fates',
 'Jugdral',
 'Valentia',
 'Heroes',
 'Tellius',
 'Heroes',
 'Jugdr

In [354]:
universe_dict = {"Fire Emblem: Shadow Dragon": "Archanea",
                    "Fire Emblem: New Mystery of the Emblem": "Archanea",
                    "Fire Emblem: Mystery of the Emblem and New Mystery of the Emblem": "Archanea",
                    "Fire Emblem Echoes: Shadows of Valentia (Gaiden)": "Valentia",
                    "Fire Emblem: Genealogy of the Holy War": "Jugdral",
                    "Fire Emblem: Thracia 776": "Jugdral",
                    "Fire Emblem (The Blazing Blade)": "Elibe",
                    "Fire Emblem: The Binding Blade": "Elibe",
                    "Fire Emblem: The Sacred Stones": "Magvel",
                    "Fire Emblem: Path of Radiance": "Tellius",
                    "Fire Emblem: Radiant Dawn": "Tellius",
                    "Fire Emblem Awakening": "Ylisse",
                    "Fire Emblem Fates: Conquest and Fire Emblem Fates: Birthright": "Fates",
                    "Fire Emblem Heroes": "Heroes",
                    "Fire Emblem: Three Houses": "Fódlan",
                    "Tokyo Mirage Sessions ♯FE": "Tokyo"}

same_name = []
same_name_count = []
refine = []
title = []
character = []
color = []
weapon = []
movement = []
rarity = []
about = []
duo = []
harmonized = []

hp = []
atk = []
spd = []
defense = []
resis = []
total = []

adj_hp = []
adj_atk = []
adj_spd = []
adj_def = []
adj_res = []
adj_total = []

obtain = []
origin = []
universe1 = []
universe2 = []

for char in all_char:
    who = char.find("h1", {"itemprop": 'name'}).text.split(" Builds ")[0]
    
    # heroes with the same name
    same_name_counter = 1
    if char.find('th', text = "Heroes With the Same Name") is not None:
        same_name_to_add = []
        for table in char.findAll('table', {'class': 'a-table a-table a-table'}):
            to_break = False
            if table.find("th", text = "Heroes With the Same Name") is not None:
                to_break = True
                for a in char.find('table', {'class': 'a-table a-table a-table'}).findAll('a'):
                    same_name_to_add.append(a.text[1:])
                    same_name_counter += 1
            if to_break is True:
                break
        same_name.append(same_name_to_add)
    else:
        same_name.append(None)
    same_name_count.append(same_name_counter)

    # refine? - should there be a second checker?
    refine.append((char.find("th", text = "Exclusive Weapon Effect from Refining (Special Effect Type)") is not None) | (
            char.find("a", {"href": "/games/fire-emblem-heroes/archives/267867"}) is not None) | (
            char.find('h2', text = who + " Best Weapon Refine") is not None))

    # basic info
    basic_title = char.find("h3", {"id": 'hm_1'}) # in header 
    char_title = basic_title.text.split(":")
    title.append(char_title[0])
    character.append(char_title[0].split(" - ")[0])
    basic = basic_title.find_next_sibling()
    a_data = basic.tbody.findAll('a')
    try:
        col = a_data[0].text
        weap = a_data[1].text
        mov = a_data[2].text
        rar = a_data[3].text
    except IndexError:
        if who == "Wolf":
            col = a_data[0]
            weap = "Bow"
            mov = a_data[1]
            rar = a_data[2]
    color.append(col)
    weapon.append(weap)
    movement.append(mov)
    rarity.append(rar)
    about.append(basic.tbody.findAll('tr')[2]) # basically blurb about what's unique about them
    
    is_duo = char.find("th", text = "Duo Skill")
    duo.append(is_duo is not None)
    
    harm = char.find("th", text = "Harmonized Skill")
    harmonized.append(harm is not None)

    # stats
    try:
        tds = char.find('h4', text = "Stats at Lv 40").findNext("table").findAll("td")
        adj_hp.append(int(tds[0].text))
        adj_atk.append(int(tds[1].text))
        adj_spd.append(int(tds[2].text))
        adj_def.append(int(tds[3].text))
        adj_res.append(int(tds[4].text))
        adj_total.append(int(tds[5].text))
        
        hp.append(int(tds[6].text))
        atk.append(int(tds[7].text))
        spd.append(int(tds[8].text))
        defense.append(int(tds[9].text))
        resis.append(int(tds[10].text))
        total.append(int(tds[11].text))
    except AttributeError: # as of now, only applies to Kiran: Hero Summoner
        stats = char.find("h3", text = "Recommended IVs").findNext("table").findAll("td")
        to_add = []
        for i in range(0, len(stats), 3):
            start = stats[i].text.find("(")
            end = stats[i].text.find(")")
            to_add.append(stats[i].text[start + 1: end])
        hp.append(int(to_add[0]))
        atk.append(int(to_add[1]))
        spd.append(int(to_add[2]))
        defense.append(int(to_add[3]))
        resis.append(int(to_add[4]))
        total.append(sum(map(int, to_add)))
        
        adj_hp.append(None)
        adj_atk.append(None)
        adj_spd.append(None)
        adj_def.append(None)
        adj_res.append(None)
        adj_total.append(None)

# similar characters?

# roles

    # how to obtain
    if who == "Ótr":
        who = "Otr" # uses this for the rest of the doc
    if who == "Thorr":
        who = "Thórr" # uses this for the rest of the doc
    if who == "Halloween Tiki (Young)":
        who = "Halloween Tiki"
    if who == "Duo Eirika and Ephraim":
        who = "Duo Eirika"
    if who == "Fallen Tiki":
        who = "Fallen Tiki (Young)"
    if who == "Legendary Tiki":
        who = "Legendary Tiki (Young)"
    if who == "Pirate Hinoka and Camilla":
        who = "Pirate Hinoka"
    if who == "Scion Leif and Seliph":
        who = "Scion Leif"
    if who == "Masquerade Eldigan Best":
        who = "Masquerade Eldigan"
    if who == "Fallen Robin":
        who = "Fallen Robin (M)"
    if who == "Lif":
        who = "Líf" # uses this the rest of the doc
    if who == "Young Azura":
        who = "Azura (Young)"
    if who == "Gangrel Best":
        who = "Gangrel"
    if who == "Hot Springs Camilla":
        who = "Hostile Springs Camilla"
    if who == "Summer Rhys Best":
        who = "Summer Rhys"
    if who == "Bridal Tanith":
        who = "Tanith" # only refers to Tanith in this one part
    if who == "Summer Lorenz Best":
        who = "Summer Lorenz"
    if who == "Kiran (Summoner)":
        who = "Kiran: Hero Summoner" # the rest of the doc refers to him as "My Summoner (Kiran)"
    start = char.find('h3', text = "How to Get " + who)
    if who == "Kiran: Hero Summoner":
        start = char.find('h3', text = "How to Obtain " + who)
    get = start.findNext("h4")
    obtain_char = []
    while get.text != "Featured Articles":
        obtain_char.append(get.text)
        get = get.findNext('h4')
    obtain.append(obtain_char)

    # about and universes
    try:
        if len(char.find('h3', text = "Voice Actor and Illustrator Information").findNext("table").findAll('div')) > 1:
            games = []
            for div in char.find('h3', text = "Voice Actor and Illustrator Information").findNext("table").findAll('div'):
                games.append(div.text) 
            origin.append(games)
        else:
            div = char.find('h3', text = "Voice Actor and Illustrator Information").findNext("table").find('div')
            games = div.text
            origin.append(games)
            
        verse = set()
        if type(games) is list:
            for game in games:
                verse.add(universe_dict[game])
            if len(verse) == 1:
                universe1.append(verse.pop())
                universe2.append(None)
            else:
                universe1.append(verse.pop())
                universe2.append(verse.pop())
        else:
            universe1.append(universe_dict[games])
            universe2.append(None)
    except AttributeError:
        if who == "Summer Ogma":
            origin.append("Fire Emblem: Shadow Dragon")
            universe1.append("Archanea")
            universe2.append(None)
        if who == "Brave Marth":
            origin.append("Fire Emblem: Shadow Dragon")
            universe1.append("Archanea")
            universe2.append(None)
        if who == "Ninja Igrene":
            origin.append("Fire Emblem: The Binding Blade")
            universe1.append("Elibe")
            universe2.append(None)
        if who == "Kiran: Hero Summoner": # change to Kiran: Hero Summoner
            origin.append("Fire Emblem Heroes")
            universe1.append("Heroes")
            universe2.append(None)

In [355]:
dfn["Same Character"] = same_name
dfn["Same Character Count"] = same_name_count
dfn["Refine"] = refine
dfn["Title"] = title
dfn["Character Name"] = character
dfn["Color"] = color
dfn["Weapon"] = weapon
dfn["Movement"] = movement
dfn["Rarity"] = rarity
dfn["About"] = about
dfn["Duo"] = duo
dfn["Harmonized"] = harmonized

dfn["HP"] = hp
dfn["Atk"] = atk
dfn["Spd"] = spd
dfn["Def"] = defense
dfn["Res"] = resis
dfn["Total"] = total

dfn["HP_adj"] = adj_hp
dfn["Atk_adj"] = adj_atk
dfn["Spd_adj"] = adj_spd
dfn["Def_adj"] = adj_def
dfn["Res_adj"] = adj_res
dfn["Total_adj"] = adj_total

dfn["Obtain"] = obtain
dfn["Game Origin"] = origin
dfn["Universe 1"] = universe1
dfn["Universe 2"] = universe2

In [356]:
dfn

,Name,Link,Rating,Same Character,Same Character Count,Refine,Title,Character Name,Color,Weapon,...,HP_adj,Atk_adj,Spd_adj,Def_adj,Res_adj,Total_adj,Obtain,Game Origin,Universe 1,Universe 2
0,Ninja Lyn,https://game8.co/games/fire-emblem-heroes/arch...,9.9,"[Brave Lyn, Bridal Lyn, Lyn, Legendary Lyn, Va...",7,False,Lyn - Ninja-Friend Duo,Lyn,Colorless,Dagger,...,40.0,47.0,46.0,17.0,22.0,172.0,[Obtained through a Special Summoning Event!],Fire Emblem (The Blazing Blade),Elibe,None
1,Brave Hector,https://game8.co/games/fire-emblem-heroes/arch...,9.9,"[Hector, Valentine's Hector, Legendary Hector,...",5,True,Hector - Brave Warrior,Hector,Blue,Lance,...,48.0,53.0,22.0,39.0,28.0,190.0,[Obtain Through Regular Summon],Fire Emblem (The Blazing Blade),Elibe,None
2,Fallen Edelgard,https://game8.co/games/fire-emblem-heroes/arch...,9.9,"[Edelgard, Legendary Edelgard, Brave Edelgard]",4,False,Edelgard - Hegemon Husk,Edelgard,Colorless,Colorless Beast,...,45.0,56.0,14.0,44.0,39.0,198.0,[Obtain Through Regular Summon],Fire Emblem: Three Houses,Fódlan,None
3,Ninja Corrin (F),https://game8.co/games/fire-emblem-heroes/arch...,9.9,"[Adrift Corrin (F), Corrin (F), Summer Corrin ...",6,False,Corrin (F) - Nightfall Ninja Act,Corrin (F),Green,Green Tome,...,38.0,50.0,44.0,19.0,16.0,167.0,[Obtained through a Special Summoning Event!],Fire Emblem Fates: Conquest and Fire Emblem Fa...,Fates,None
4,Legendary Dimitri,https://game8.co/games/fire-emblem-heroes/arch...,9.5,"[Dimitri, Brave Dimitri, Fallen Dimitri]",4,False,Dimitri - Savior King,Dimitri,Blue,Lance,...,40.0,57.0,41.0,37.0,18.0,193.0,[Obtained through a Special Summoning Event!],Fire Emblem: Three Houses,Fódlan,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,Kiran,https://game8.co/games/fire-emblem-heroes/arch...,NaN,None,1,False,Kiran - Hero Summoner,Kiran,Colorless,Colorless Tome,...,NaN,NaN,NaN,NaN,NaN,NaN,[],Fire Emblem Heroes,Heroes,None
766,Valentine's Lucina,https://game8.co/games/fire-emblem-heroes/arch...,NaN,"[Brave Lucina, Lucina, Legendary Lucina, Sprin...",5,False,Lucina - Future Fondness,Lucina,Green,Axe,...,41.0,55.0,45.0,37.0,31.0,209.0,[Obtained through a Special Summoning Event!],Fire Emblem Awakening,Ylisse,None
767,Valentine's Robin (F),https://game8.co/games/fire-emblem-heroes/arch...,NaN,"[Legendary Robin (F), Robin (F), Summer Robin ...",5,False,Robin - Tactful Deliverer,Robin,Colorless,Colorless Tome,...,42.0,55.0,16.0,39.0,42.0,194.0,[Obtained through a Special Summoning Event!],Fire Emblem Awakening,Ylisse,None
768,Valentine's Owain,https://game8.co/games/fire-emblem-heroes/arch...,NaN,[Owain],2,False,Owain - Devoted Defender,Owain,Blue,Lance,...,41.0,52.0,42.0,31.0,21.0,187.0,[Obtained through a Special Summoning Event!],Fire Emblem Awakening,Ylisse,None


In [358]:
last_update = time.strftime("%m-%y")
dfn.to_csv("hero_data_g8_" + last_update + ".csv", encoding='utf-8', index=False)

In [ ]:
# get links for the roles and look through characters

# separate thing to get these things from 'how to obtain'
grand hero battle
story
tempest trial
regular summon
special summon

# fix up weapons column